In [1]:
import kodsp as dsp
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import chain
from rank_bm25 import BM25Okapi

openai_key = 'YOUR_API_KEY'

Not loading Cohere because it is not installed.
Not loading Cohere because it is not installed.


In [3]:
def convert_data_to_dsp_v1(data_type):
    dsp_examples = []
    with open(f"./KorWikiTQ_ko_{data_type}.json", "r") as json_file:
        korwikitq = json.load(json_file)['data']
    for idx, datum in enumerate(korwikitq):
        dsp_examples.append(dsp.Example(question=datum['QAS']['question'], 
                                        answer=datum['QAS']['answer']))
    return dsp_examples

In [4]:
def convert_list_to_df(table_list):
    table_df = pd.DataFrame(table_list)
    table_df = table_df.rename(columns=table_df.iloc[0])
    table_df = table_df.drop(table_df.index[0])
    table_df.reset_index(drop=True, inplace=True)
    table_df = table_df.astype('str')
    table_df.columns = ["nan" if col == None else col for col in list(table_df.columns)]
    return table_df

In [5]:
def convert_table_to_text(table_lst):
    table_df = convert_list_to_df(table_lst)
    table = {}
    table['columns'] = table_df.columns.tolist()
    table['values'] = table_df.values.tolist()
    return json.dumps(table, ensure_ascii = False)

# LM

In [6]:
lm = dsp.GPT3(model='gpt-3.5-turbo', api_key=openai_key, model_type='chat')

dsp.settings.configure(lm=lm)

# RM
> 빠른 실험 및 쉬운 재현을 위해 bm25를 사용했습니다.

In [7]:
def retrieve_bm25(question, k):
    return [txt_to_json_dict[' '.join(top_n_table)] for top_n_table in bm25.get_top_n(question.split(" "), corpus, n=k)]

### Corpus 구축
> Retrieval에 활용할 Corpus를 준비합니다. 해당 노트북은 DSP의 작동 방식을 보여주기 위한 튜토리얼 노트북이기 때문에 korwikitq dev 데이터에 포함된 표만을 사용하여 아주 작은 표 corpus를 구축했습니다.

In [8]:
with open(f"./KorWikiTQ_ko_dev.json", "r") as json_file:
    korwikitq = json.load(json_file)['data']

corpus = []
txt_to_json_dict = defaultdict(str)

for datum in korwikitq:
    table_txt = " ".join(list(chain(*datum['TBL'])))
    if table_txt not in txt_to_json_dict:
        corpus.append(table_txt.split(" "))
        txt_to_json_dict[table_txt] = convert_table_to_text(datum['TBL'])

In [9]:
# custom 예제를 위한 table을 하나 추가합니다!

dsba_table = [['이름', '연구실', '학기', '생일', '좋아하는 드라마'],
              ['마민정', 'DSBA', '1', '3월 4일', '상속자들'],
              ['이지윤', 'DSBA', '2', '9월 23일', '별에서 온 그대'],
              ['박새란', 'DSBA', '2', '10월 5일', '꽃보다 남자'],
              ['오수지', 'DSBA', '4', '12월 1일', '구가의 서'],
              ['김선우', 'DSBA', '4', '4월 12일', '무빙']]

dsba_txt = " ".join(list(chain(*dsba_table)))
corpus.append(dsba_txt.split(" "))
txt_to_json_dict[dsba_txt] = convert_table_to_text(dsba_table)

bm25 = BM25Okapi(corpus)

# Data

In [10]:
train = convert_data_to_dsp_v1(data_type='train')
dev = convert_data_to_dsp_v1(data_type='dev')

In [11]:
# 원하는 task에 맞게 prefix, desc, instructions만 간단하게 수정해주면 끝입니다!

Question = dsp.Type(prefix="질문: ", desc="${표와 관련된 질문}")
Answer = dsp.Type(prefix="정답: ", desc="${표에서 찾아낸 질문에 대한 답}", format=dsp.format_answers)
Table = dsp.Type(
    prefix="표:\n",
    desc="${JSON 형식으로 주어진 표}",
    format=dsp.passages2text
)

SearchRationale = dsp.Type(
    prefix="근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 질문에 답하기 위해 우리가 가장 먼저 찾아야 하는 것은",
    desc="${정답을 찾기 위해 가장 필요한 정보}"
)

SearchQuery = dsp.Type(
    prefix="검색문: ",
    desc="${필요한 정보를 찾기 위한 간단한 질문}"
)

rewrite_template = dsp.Template(
    instructions="질문에 답하는 데 도움이 될 수 있는 검색문을 작성하세요.",
    question=Question(), rationale=SearchRationale(), query=SearchQuery()
)

CondenseRationale = dsp.Type(
    prefix="근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다.",
    desc="${표에서 찾을 수 있는 정답에 대한 중요한 단서를 이용한 짧은 추론}"
)

hop_template = dsp.Template(
    instructions=rewrite_template.instructions,
    context=Table(), question=Question(), rationale=CondenseRationale(), query=SearchQuery()
)

Rationale = dsp.Type(
    prefix="근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다.",
    desc="${올바른 정답을 찾기 위한 단계별 추론}"
)

qa_template_v3 = dsp.Template(instructions="JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.",
                              context=Table(),
                              question=Question(),
                              rationale=Rationale(),
                              answer=Answer())

# Demonstrate-Search-Predict

In [22]:
from dsp.utils import deduplicate

@dsp.transformation
def multihop_search_v1(example: dsp.Example, max_hops=2, k=2) -> dsp.Example:
    example.context = []
    
    for hop in range(max_hops):
        # Generate a query based
        template = rewrite_template if hop == 0 else hop_template
        example, completions = dsp.generate(template)(example, stage=f'h{hop}')

        # Retrieve k results based on the query generated
        passages = retrieve_bm25(completions.query, k=k)

        # Update the context by concatenating old and new passages
        example.context = deduplicate(example.context + passages)

    return example

def multihop_QA_v1(question: str) -> str:
    demos = dsp.sample(train, k=5)
    x = dsp.Example(question=question, demos=demos)
    
    x = multihop_search_v1(x)
    example, completions = dsp.generate(qa_template_v3)(x, stage='qa')

    return completions.answer

In [23]:
# DSP는 Search 단계에서 라벨링된 데이터 없이도 LM-RM의 상호 작용을 통해 여러 번의 hop이 가능합니다.
# 이 과정을 직접 확인하기 위해 임의로 2개의 표를 참고해야만 답을 낼 수 있는 질문을 만들었습니다.

multihop_QA_v1("DSBA 연구실 오수지가 좋아하는 드라마가 방영한 연도는?")

'오수지가 좋아하는 드라마인 "구가의 서"는 2013년에 방영되었습니다.'

In [24]:
lm.inspect_history(n=3)





질문에 답하는 데 도움이 될 수 있는 검색문을 작성하세요.

---

출력 형식은 다음과 같습니다.

질문: ${표와 관련된 질문}
근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 질문에 답하기 위해 우리가 가장 먼저 찾아야 하는 것은 ${정답을 찾기 위해 가장 필요한 정보}
검색문: ${필요한 정보를 찾기 위한 간단한 질문}

---

질문: DSBA 연구실 오수지가 좋아하는 드라마가 방영한 연도는?
근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 질문에 답하기 위해 우리가 가장 먼저 찾아야 하는 것은오수지가 좋아하는 드라마의 방영 연도입니다.
검색문: "오수지가 좋아하는 드라마 방영 연도"







질문에 답하는 데 도움이 될 수 있는 검색문을 작성하세요.

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다. ${표에서 찾을 수 있는 정답에 대한 중요한 단서를 이용한 짧은 추론}

검색문: ${필요한 정보를 찾기 위한 간단한 질문}

---

표:
[1] «{"columns": ["이름", "연구실", "학기", "생일", "좋아하는 드라마"], "values": [["마민정", "DSBA", "1", "3월 4일", "상속자들"], ["이지윤", "DSBA", "2", "9월 23일", "별에서 온 그대"], ["박새란", "DSBA", "2", "10월 5일", "꽃보다 남자"], ["오수지", "DSBA", "4", "12월 1일", "구가의 서"], ["김선우", "DSBA", "4", "4월 12일", "무빙"]]}»
[2] «{"columns": ["년도", "수상", "분류", "결과", "작품"], "values": [["2004", "영국인 아카데미 영화 어워드", "Best n